In [9]:
import os
from functools import wraps
from datetime import datetime

class CustomFileNotFoundError(OSError):
    def __init__(self, filepath):
        self.filepath = filepath
        super().__init__(f"Файл не знайдено: {filepath}")

class FileCorruptedError(OSError):
    def __init__(self, filepath, reason=""):
        self.filepath = filepath
        self.reason = reason
        message = f"Файл пошкоджено: {filepath}"
        if reason:
            message += f" ({reason})"
        super().__init__(message)

def logged(target_exception=OSError):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            try:
                with open("operations.log", "a", encoding="utf-8") as log_file:
                    log_file.write(f"{timestamp} - {func.__name__} - INFO - Виконання методу {func.__name__}\n")
                result = func(*args, **kwargs)
                with open("operations.log", "a", encoding="utf-8") as log_file:
                    log_file.write(f"{timestamp} - {func.__name__} - INFO - Метод {func.__name__} виконано успішно\n")
                return result
            except target_exception as e:
                with open("operations.log", "a", encoding="utf-8") as log_file:
                    log_file.write(f"{timestamp} - {func.__name__} - ERROR - Помилка {type(e).__name__}: {str(e)}\n")
                raise
        return wrapper
    return decorator

class TextFileHandler:
    def __init__(self, filepath):
        self.filepath = filepath
        if not os.path.exists(filepath):
            raise CustomFileNotFoundError(filepath)
        if not os.path.isfile(filepath):
            raise FileCorruptedError(filepath, "Це не файл, а папка")

    @logged()
    def read(self):
        try:
            with open(self.filepath, 'r', encoding='utf-8') as file:
                return file.read()
        except PermissionError:
            raise FileCorruptedError(self.filepath, "Немає прав на читання")
        except UnicodeDecodeError:
            raise FileCorruptedError(self.filepath, "Помилка декодування тексту")
        except OSError as e:
            raise FileCorruptedError(self.filepath, str(e))

    @logged()
    def write(self, content):
        try:
            with open(self.filepath, 'w', encoding='utf-8') as file:
                file.write(content)
        except PermissionError:
            raise FileCorruptedError(self.filepath, "Немає прав на запис")
        except OSError as e:
            raise FileCorruptedError(self.filepath, str(e))

    @logged()
    def append(self, content):
        try:
            with open(self.filepath, 'a', encoding='utf-8') as file:
                file.write(content)
        except PermissionError:
            raise FileCorruptedError(self.filepath, "Немає прав на запис")
        except OSError as e:
            raise FileCorruptedError(self.filepath, str(e))

def main():
    print("Демонстрація роботи класу TextFileHandler\n")
    
    if os.path.exists("operations.log"):
        os.remove("operations.log")
    
    test_file = "test_file.txt"
    with open(test_file, 'w', encoding='utf-8') as f:
        f.write("Початковий вміст файлу.\n")
    
    print("1. Робота з існуючим файлом:")
    try:
        handler = TextFileHandler(test_file)
        print("   Читання файлу...")
        content = handler.read()
        print(f"   Вміст: {content.strip()}")
        
        print("   Запис нового вмісту...")
        handler.write("Новий текст у файлі.\n")
        
        print("   Дописування тексту...")
        handler.append("Додатковий рядок.\n")
        
        print("   Читання оновленого файлу...")
        content = handler.read()
        print("   Оновлений вміст:")
        for line in content.split('\n'):
            if line:
                print(f"     {line}")
    except Exception as e:
        print(f"   Помилка: {e}")
    
    print("\n2. Спроба відкрити неіснуючий файл:")
    try:
        handler = TextFileHandler("nonexistent_file.txt")
    except CustomFileNotFoundError as e:
        print(f"   Перехоплено виняток: {e}")
    
    print("\n3. Перегляд лог-файлу:")
    if os.path.exists("operations.log"):
        print("   Вміст operations.log:")
        print("   " + "-" * 50)
        with open("operations.log", 'r', encoding='utf-8') as log_file:
            for line in log_file:
                print("   " + line.rstrip())
        print("   " + "-" * 50)
    else:
        print("   Лог-файл не створено")
    
    print("\n4. Очищення тестових файлів...")
    if os.path.exists(test_file):
        os.remove(test_file)
        print("   test_file.txt видалено")
    if os.path.exists("operations.log"):
        os.remove("operations.log")
        print("   operations.log видалено")
    
    print("\nДемонстрацію завершена")

if __name__ == "__main__":
    main()


Демонстрація роботи класу TextFileHandler

1. Робота з існуючим файлом:
   Читання файлу...
   Вміст: Початковий вміст файлу.
   Запис нового вмісту...
   Дописування тексту...
   Читання оновленого файлу...
   Оновлений вміст:
     Новий текст у файлі.
     Додатковий рядок.

2. Спроба відкрити неіснуючий файл:
   Перехоплено виняток: Файл не знайдено: nonexistent_file.txt

3. Перегляд лог-файлу:
   Вміст operations.log:
   --------------------------------------------------
   2025-12-03 12:41:35 - read - INFO - Виконання методу read
   2025-12-03 12:41:35 - read - INFO - Метод read виконано успішно
   2025-12-03 12:41:35 - write - INFO - Виконання методу write
   2025-12-03 12:41:35 - write - INFO - Метод write виконано успішно
   2025-12-03 12:41:35 - append - INFO - Виконання методу append
   2025-12-03 12:41:35 - append - INFO - Метод append виконано успішно
   2025-12-03 12:41:35 - read - INFO - Виконання методу read
   2025-12-03 12:41:35 - read - INFO - Метод read виконано успі